In [1]:
# AutoDL官方学术资源加速
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [2]:
import sys
import os

# 添加项目根目录到Python路径
project_root = "/home/cuipeng/Gemma"
sys.path.append(project_root)

# 现在可以正常导入src下的模块
from src.core.model.model_initializer import initialize_model_and_tokenizer
from src.core.utils.model_utils import generate_response, apply_chat_template
from src.core.solvers.l2m import L2MSolver
from src.core.solvers.self_verification import SelfVerifier

In [3]:
# self_verification

import torch # type: ignore
from transformers import AutoTokenizer, AutoModelForCausalLM # type: ignore
from transformers import BitsAndBytesConfig # type: ignore
from peft import PeftModel  # type: ignore # 导入PeftModel用于加载微调模型

In [4]:
class EnhancedSolver:
    def __init__(self, l2m_solver, verifier):
        """
        初始化增强型求解器
        Args:
            l2m_solver: 已初始化的 L2MSolver 实例
            verifier: 已初始化的 SelfVerifier 实例
        """
        if l2m_solver is None or verifier is None:
            raise ValueError("必须提供已初始化的 L2MSolver 和 SelfVerifier 实例")
        
        self.l2m = l2m_solver
        self.verifier = verifier
        
        # 添加系统提示词
        self.system_prompt = """
        你是一个专业的问题解决专家。你的任务是:
        1. 分解复杂问题
        2. 逐步解决子问题
        3. 验证和改进答案
        4. 整合最终解决方案

        请确保答案:
        - 准确完整
        - 逻辑清晰
        - 层次分明
        - 易于理解
        """

    def solve_complex_question(self, question):
        """使用增强的解决方案处理复杂问题"""
        print("开始处理问题:", question)
        
        # 1. 使用L2M获取初始解答
        print("\n1. 分解并解决问题...")
        l2m_result = self.l2m.solve(question) # l2m_result是一个字典
        
        # 2. 对每个子问题的解答进行验证和改进
        print("\n2. 验证和改进子问题解答...")
        verified_solutions = []
        improved_context = ""
        
        for sub_solution in l2m_result["sub_solutions"]: # 每个sub_solution都类似：{'question': , 'solution': }
            print(f"\n处理子问题: {sub_solution['question']}")
            verified = self.verifier.verify_and_improve(
                sub_solution["question"],
                sub_solution["solution"]
            )
            
            verified_solutions.append({
                "question": sub_solution["question"],
                "original_solution": sub_solution["solution"],
                "verified_solution": verified["improved_answer"],
                "verification_process": {
                    "first_verification": verified["first_verification"],
                    "final_verification": verified["final_verification"]
                }
            })
            improved_context += f"\n问题：{sub_solution['question']}\n答案：{verified['improved_answer']}\n"
        
        # 3. 基于改进后的答案重新生成最终答案
        print("\n3. 生成最终答案...")
        dialogue = [
            {
                "role": "system",
                "content": self.system_prompt
            },
            {
                "role": "user",
                "content": f"""基于以下解答过程，生成完整答案：

解答过程：
{improved_context}

原始问题：{question}

要求：
1. 综合所有子问题的答案
2. 保持逻辑连贯性
3. 突出重点内容
4. 语言简洁清晰"""
            }
        ]
        
        prompt = apply_chat_template(dialogue)
        new_final_answer = generate_response(
            self.l2m.model,
            self.l2m.tokenizer,
            prompt,
            max_new_tokens=1536,
            temperature=0.5
        )
        
        # 4. 对最终答案进行验证和改进
        print("\n4. 验证和改进最终答案...")
        final_verified = self.verifier.verify_and_improve(
            question,
            new_final_answer
        )
        
        return {
            "original_question": question,
            "sub_solutions": verified_solutions,
            "final_answer": final_verified["improved_answer"],
            "solution_process": {
                "decomposition": l2m_result["sub_solutions"],
                "improved_context": improved_context,
                "final_verification": final_verified
            }
        }

    def analyze_verification_results(self, result):
        """分析验证结果，提供改进建议"""
        analysis = {
            "sub_problems": [],
            "final_answer": {
                "improvement_level": None,
                "key_improvements": [],
                "remaining_issues": []
            }
        }
        
        # 分析子问题的改进
        for sub_sol in result["sub_solutions"]:
            analysis["sub_problems"].append({
                "question": sub_sol["question"],
                "verification_analysis": self._analyze_verification(
                    sub_sol["verification_process"]
                )
            })
        
        # 分析最终答案的改进
        final_verification = result["solution_process"]["final_verification"]
        analysis["final_answer"].update(
            self._analyze_verification(final_verification)
        )
        
        return analysis

    def _analyze_verification(self, verification_data):
        """分析单个验证过程"""
        return {
            "initial_issues": self._extract_issues(verification_data["first_verification"]),
            "resolved_issues": self._extract_issues(verification_data["final_verification"]),
            "improvement_summary": self._compare_verifications(
                verification_data["first_verification"],
                verification_data["final_verification"]
            )
        }

    def _extract_issues(self, verification):
        """从验证结果中提取问题"""
        # 这里可以添加更详细的解析逻辑
        return verification

    def _compare_verifications(self, first, final):
        """比较两次验证结果的差异"""
        # 这里可以添加更详细的比较逻辑
        return {
            "improved_aspects": [],
            "remaining_issues": []
        }

In [5]:
# 使用示例
if __name__ == "__main__":
    # 初始化模型和tokenizer
    model_path = "google/gemma-2-9b"
    cache_dir = "/root/autodl-tmp/gemma"
    # lora_path = "/root/autodl-tmp/models/stage1/gemma-base-zh-final"
    lora_path = None
    
    model, tokenizer = initialize_model_and_tokenizer(
        model_path=model_path,
        cache_dir=cache_dir,
        lora_path=lora_path,
        use_quantization=True  # 确保使用量化
    )
    
    # 初始化L2M求解器和验证器
    l2m_solver = L2MSolver(model, tokenizer)
    verifier = SelfVerifier(model, tokenizer)
    
    # 初始化增强求解器
    solver = EnhancedSolver(
        l2m_solver=l2m_solver,
        verifier=verifier
    )
    
    # 测试问题
    question = "请解释量子计算机的工作原理及其潜在应用。"
    
    # 执行求解过程
    result = solver.solve_complex_question(question)
    
    # 输出结果
    print("\n最终答案:", result["final_answer"])
    print("\n解答过程分析:", solver.analyze_verification_results(result))

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

/root/miniconda3/envs/Gemma/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:676: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
The 'batch_size' attribute of HybridCache is deprecated and will be removed in v4.49. Use the more precisely named 'self.max_batch_size' attribute instead.


开始处理问题: 请解释量子计算机的工作原理及其潜在应用。

1. 分解并解决问题...

2. 验证和改进子问题解答...

处理子问题: ")

处理子问题: for i, subproblem in enumerate(decompose_problem(question), start=1):

处理子问题: print(f"{i}. {subproblem}")

处理子问题: if __name__ == "__main__":

处理子问题: main()
